In [2]:
import pandas as pd

In [15]:
df = pd.read_csv('ab_data - ab_data.csv')
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


In [17]:
df.timestamp = pd.to_datetime(df.timestamp)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   user_id       294478 non-null  int64         
 1   timestamp     294478 non-null  datetime64[ns]
 2   group         294478 non-null  object        
 3   landing_page  294478 non-null  object        
 4   converted     294478 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 11.2+ MB


In [19]:
# проверка на уникальность каждого столбца
df.nunique()

user_id         290584
timestamp       294478
group                2
landing_page         2
converted            2
dtype: int64

In [56]:
# вывести  дубли в user_id
duplicated_user_id = df[df.duplicated(['user_id'],keep = False)].sort_values('user_id')
duplicated_user_id

,user_id,timestamp,group,landing_page,converted
230259,630052,2017-01-17 01:16:05.208766,treatment,new_page,0
213114,630052,2017-01-07 12:25:54.089486,treatment,old_page,1
22513,630126,2017-01-14 13:35:54.778695,treatment,old_page,0
251762,630126,2017-01-19 17:16:00.280440,treatment,new_page,0
183371,630137,2017-01-20 02:08:49.893878,control,old_page,0
...,...,...,...,...,...
142354,945703,2017-01-08 19:40:51.169351,control,new_page,0
186960,945797,2017-01-13 17:23:21.750962,control,old_page,0
40370,945797,2017-01-11 03:04:49.433736,control,new_page,1
165143,945971,2017-01-16 10:09:18.383183,control,old_page,0


In [60]:
# посчитать количество дублей чтобы понять что с ними делать
duplicated_user_id = df[df.duplicated(['user_id'],keep = False)]['user_id'].nunique()
duplicated_user_id

3894

In [30]:
# общее количество user_id в DF
all_user_id = df.user_id.nunique()
all_user_id

290584

In [62]:
# % дублей
duplicated_user_id/all_user_id * 100

1.3400600170690746

In [36]:
# дропаем дубли
df_without_dublicates = df.drop_duplicates('user_id',keep = False)

In [43]:
# проверяем что контрольной группе не показалась новая страница и что тестовой группе не показалась старая страница
print(df_without_dublicates[(
    df_without_dublicates['group'] == 'treatment') & 
    (df_without_dublicates['landing_page'] == 'old_page')].shape)
print(df_without_dublicates[(
    df_without_dublicates['group'] == 'control') & 
    (df_without_dublicates['landing_page'] == 'old_page')].shape)
print(df_without_dublicates[(
    df_without_dublicates['group'] == 'treatment') & 
    (df_without_dublicates['landing_page'] == 'new_page')].shape)
print(df_without_dublicates[(
    df_without_dublicates['group'] == 'control') & 
    (df_without_dublicates['landing_page'] == 'new_page')].shape)

(0, 5)
(143293, 5)
(143397, 5)
(0, 5)


In [46]:
# дсначала смотрим среднее потом общее количество
print(df_without_dublicates.groupby('group').converted.mean())
print(df_without_dublicates.group.value_counts())

group
control      0.120173
treatment    0.122938
Name: converted, dtype: float64
treatment    143397
control      143293
Name: group, dtype: int64


In [49]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable


In [50]:
from statsmodels.stats.weightstats import ztest

In [51]:
statistics,p_value = ztest(df_without_dublicates[df_without_dublicates['group'] == 'treatment'].converted.values,
                           df_without_dublicates[df_without_dublicates['group'] == 'control'].converted.values)

In [52]:
print(f'statistics: {statistics}')
print(f'p_value: {p_value}')


statistics: 2.265364406392722
p_value: 0.02349032668040717
